In [1]:
import fsspec
import xarray as xr

import plotly.graph_objects as go
import pandas as pd
from tqdm import tqdm

In [2]:
filepath = 'https://power-analysis-ready-datastore.s3.amazonaws.com/power_901_monthly_meteorology_utc.zarr'
filepath_mapped = fsspec.get_mapper(filepath)

ds = xr.open_zarr(store=filepath_mapped, consolidated=True)
ds

<xarray.Dataset>
Dimensions:          (time: 492, lat: 361, lon: 576)
Coordinates:
  * lat              (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * lon              (lon) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * time             (time) datetime64[ns] 1981-01-31 1981-02-28 ... 2021-12-31
Data variables: (12/94)
    CDD0             (time, lat, lon) float64 ...
    CDD10            (time, lat, lon) float64 ...
    CDD18_3          (time, lat, lon) float64 ...
    DISPH            (time, lat, lon) float64 ...
    EVLAND           (time, lat, lon) float64 ...
    EVPTRNS          (time, lat, lon) float64 ...
    ...               ...
    WS50M_MAX_AVG    (time, lat, lon) float64 ...
    WS50M_MIN        (time, lat, lon) float64 ...
    WS50M_MIN_AVG    (time, lat, lon) float64 ...
    WS50M_RANGE      (time, lat, lon) float64 ...
    WS50M_RANGE_AVG  (time, lat, lon) float64 ...
    Z0M              (time, lat, lon) float64 ...
Attributes: (12/37)
    acknowledgement:            The Prediction of Worldwide Energy Resources ...
    comment:                    Version 9.0.1 includes source data resolution.
    conventions:                CF-1.8, ACDD-1.3
    creator_email:              bradley.macpherson@nasa.gov
    creator_name:               Bradley Macpherson
    date_created:               2022-04-11T12:20:00.000000
    ...                         ...
    time_coverage_end:          2021-12-31T00:00:00
    time_coverage_resolution:   P1M
    time_coverage_start:        1981-01-01T00:00:00
    time_standard:              Universal Time Coordinated (UTC)
    title:                      POWER Monthly Meteorology Time Series
    version:                    POWER Version 9.0.1

In [3]:
# SOURCE = /power_901_monthly_meteorology_utc.zarr/
# TSURF -------
# long_name     : Surface Temperature of Land and Snow
# standard_name : Surface_Temperature_of_Land_and_Snow
# units         : K
# valid_max     : 350.0
# valid_min     : 150.0
# valid_range   : 150.0, 350.0]

# GWETTOP -----
# long_name     : Surface Soil Wetness
# standard_name : Surface_Soil_Wetness
# units         : 1
# valid_max     : 1.0
# valid_min     : 0.0
# valid_range   : [0.0, 1.0]

# PRECSNO --------
# long_name     : Snow Precipitation
# standard_name : Snow_Precipitation
# units         : kg m-2 s-1
# valid_max     : 0.0005
# valid_min     : 0.0
# valid_range   : [0.0, 0.0005]

# RH2M -------
# long_name     : Relative Humidity at 2 Meters
# standard_name : Relative_Humidity_at_2_Meters
# units         : %
# valid_max     : 100.0
# valid_min     : 0.0
# valid_range   : [0.0, 100.0]

# WS2M -------
# long_name     : Wind Speed at 2 Meters
# standard_name : Wind_Speed_at_2_Meters
# units         : m/s
# valid_max     : 50.0
# valid_min     : 0.0
# valid_range   : [0.0, 50.0]

# Source = /power_901_daily_precipitation_utc.zarr/
# PRECIPITATIONCAL ------
# long_name : The accumulated precipitation from all available infrared (IR) and microwave (MW) sources.
# units     : mm/day

## Select Columns of Interest

In [4]:
selected = ds[["TSURF", "GWETTOP", "PRECSNO", "RH2M", "WS2M"]]
selected

<xarray.Dataset>
Dimensions:  (time: 492, lat: 361, lon: 576)
Coordinates:
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * time     (time) datetime64[ns] 1981-01-31 1981-02-28 ... 2021-12-31
Data variables:
    TSURF    (time, lat, lon) float64 ...
    GWETTOP  (time, lat, lon) float64 ...
    PRECSNO  (time, lat, lon) float64 ...
    RH2M     (time, lat, lon) float64 ...
    WS2M     (time, lat, lon) float64 ...
Attributes: (12/37)
    acknowledgement:            The Prediction of Worldwide Energy Resources ...
    comment:                    Version 9.0.1 includes source data resolution.
    conventions:                CF-1.8, ACDD-1.3
    creator_email:              bradley.macpherson@nasa.gov
    creator_name:               Bradley Macpherson
    date_created:               2022-04-11T12:20:00.000000
    ...                         ...
    time_coverage_end:          2021-12-31T00:00:00
    time_coverage_resolution:   P1M
    time_coverage_start:        1981-01-01T00:00:00
    time_standard:              Universal Time Coordinated (UTC)
    title:                      POWER Monthly Meteorology Time Series
    version:                    POWER Version 9.0.1

## Select Data from 2015-2021
We also drop NA values which are readings of the sea.

In [5]:
all_data = selected.to_dataframe().reset_index()
all_data

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
0,1981-01-31,-90.0,-180.000,NaN,1.0,0.000000,91.3750,2.257812
1,1981-01-31,-90.0,-179.375,NaN,1.0,0.000000,91.3750,2.257812
2,1981-01-31,-90.0,-178.750,NaN,1.0,0.000000,91.3750,2.265625
3,1981-01-31,-90.0,-178.125,NaN,1.0,0.000000,91.3750,2.273438
4,1981-01-31,-90.0,-177.500,NaN,1.0,0.000000,91.3750,2.281250
...,...,...,...,...,...,...,...,...
102304507,2021-12-31,90.0,176.875,NaN,1.0,0.000005,89.9375,4.117188
102304508,2021-12-31,90.0,177.500,NaN,1.0,0.000005,89.9375,4.125000
102304509,2021-12-31,90.0,178.125,NaN,1.0,0.000005,89.9375,4.132812
102304510,2021-12-31,90.0,178.750,NaN,1.0,0.000005,89.9375,4.140625


In [6]:
all_data.describe()

,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
count,1.023045e+08,1.023045e+08,2.896699e+07,1.023045e+08,1.023045e+08,1.023045e+08,1.023045e+08
mean,0.000000e+00,-3.125000e-01,2.826836e+02,8.896880e-01,4.425585e-06,8.174603e+01,5.157919e+00
std,5.210566e+01,1.039229e+02,1.804416e+01,2.256860e-01,9.143772e-06,1.447399e+01,2.322250e+00
min,-9.000000e+01,-1.800000e+02,2.200938e+02,7.812500e-03,0.000000e+00,5.875000e+00,0.000000e+00
25%,-4.500000e+01,-9.015625e+01,2.710703e+02,9.140625e-01,0.000000e+00,7.825000e+01,3.585938e+00
50%,-1.797510e-13,-3.125000e-01,2.861875e+02,1.000000e+00,0.000000e+00,8.331250e+01,5.195312e+00
75%,4.500000e+01,8.953125e+01,2.978828e+02,1.000000e+00,7.629395e-06,9.081250e+01,6.679688e+00
max,9.000000e+01,1.793750e+02,3.175000e+02,1.000000e+00,6.561279e-04,1.000000e+02,1.864844e+01


In [7]:
all_data.dropna(inplace=True)
all_data.reset_index(inplace=True)
all_data

,index,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
0,34806,1981-01-31,-60.0,-26.250,275.656250,1.000000,0.000015,92.8125,6.421875
1,35378,1981-01-31,-59.5,-28.750,275.781250,1.000000,0.000023,92.8125,6.304688
2,35379,1981-01-31,-59.5,-28.125,275.781250,0.953125,0.000023,92.9375,6.281250
3,35380,1981-01-31,-59.5,-27.500,275.750000,0.914062,0.000023,93.0000,6.304688
4,35381,1981-01-31,-59.5,-26.875,275.664062,0.890625,0.000015,93.0625,6.382812
...,...,...,...,...,...,...,...,...,...
28966987,102297252,2021-12-31,84.0,-37.500,244.812500,0.984375,0.000005,97.1250,5.078125
28966988,102297253,2021-12-31,84.0,-36.875,244.812500,0.984375,0.000005,97.3750,5.117188
28966989,102297254,2021-12-31,84.0,-36.250,244.812500,0.984375,0.000006,97.6250,5.156250
28966990,102297255,2021-12-31,84.0,-35.625,244.812500,0.992188,0.000006,97.7500,5.195312


In [8]:
all_data.drop(columns=["index"], inplace=True)

## Convert Kelvin to Fahrenheit

In [9]:
all_data["F_TSURF"] = (9/5) * (all_data["TSURF"] - 273) + 32

all_data

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M,F_TSURF
0,1981-01-31,-60.0,-26.250,275.656250,1.000000,0.000015,92.8125,6.421875,36.781250
1,1981-01-31,-59.5,-28.750,275.781250,1.000000,0.000023,92.8125,6.304688,37.006250
2,1981-01-31,-59.5,-28.125,275.781250,0.953125,0.000023,92.9375,6.281250,37.006250
3,1981-01-31,-59.5,-27.500,275.750000,0.914062,0.000023,93.0000,6.304688,36.950000
4,1981-01-31,-59.5,-26.875,275.664062,0.890625,0.000015,93.0625,6.382812,36.795313
...,...,...,...,...,...,...,...,...,...
28966987,2021-12-31,84.0,-37.500,244.812500,0.984375,0.000005,97.1250,5.078125,-18.737500
28966988,2021-12-31,84.0,-36.875,244.812500,0.984375,0.000005,97.3750,5.117188,-18.737500
28966989,2021-12-31,84.0,-36.250,244.812500,0.984375,0.000006,97.6250,5.156250,-18.737500
28966990,2021-12-31,84.0,-35.625,244.812500,0.992188,0.000006,97.7500,5.195312,-18.737500


## Get US Counties Coordinates

In [10]:
us_counties = pd.read_csv("./us_counties.csv")
us_counties

,fips_code,name,lng,lat
0,1059,Franklin,-87.843283,34.442381
1,13111,Fannin,-84.319296,34.864126
2,19109,Kossuth,-94.206898,43.204140
3,40115,Ottawa,-94.810589,36.835878
4,42115,Susquehanna,-75.800905,41.821277
...,...,...,...,...
3228,12029,Dixie,-83.158705,29.608068
3229,18017,Cass,-86.346207,40.761660
3230,26091,Lenawee,-84.066412,41.894694
3231,72003,Aguada,-67.175247,18.360392


## Get Counties of Interest

In [17]:
latitudes = [35.5,36.0,26.5,28.0,29.5,29.0,46.5,48.0,43.5,42.5,40.5,43.5]
longitudes = [80.625,81.25,80.625,81.25,98.75,95.625,120.625,121.875,72.5,71.25,-93.75,-102.5]

In [18]:
all_in_lat = all_data[all_data['lat'].isin(latitudes)]
all_of_interest = all_in_lat[all_in_lat['lon'].isin(longitudes)]

In [19]:
all_of_interest

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M,F_TSURF
22283,1981-01-31,26.5,-102.500,282.843750,0.460938,0.000008,65.5000,2.617188,49.718750
22437,1981-01-31,26.5,71.250,289.304688,0.187500,0.000000,34.1250,2.281250,61.348438
22439,1981-01-31,26.5,72.500,289.265625,0.171875,0.000000,35.5000,2.156250,61.278125
22452,1981-01-31,26.5,80.625,286.992188,0.500000,0.000000,59.5625,1.429688,57.185938
22453,1981-01-31,26.5,81.250,287.304688,0.468750,0.000000,58.9375,1.484375,57.748438
...,...,...,...,...,...,...,...,...,...
28943124,2021-12-31,48.0,81.250,262.484375,0.445312,0.000008,81.1250,3.031250,13.071875
28943147,2021-12-31,48.0,95.625,255.289062,0.429688,0.000001,74.7500,2.296875,0.120313
28943152,2021-12-31,48.0,98.750,252.585938,0.601562,0.000001,76.1250,2.804688,-4.745313
28943187,2021-12-31,48.0,120.625,252.984375,0.757812,0.000004,94.1875,4.507812,-4.028125


In [20]:
all_of_interest.to_csv("12_counties_data.csv")